In [1]:
import pandas as pd
import numpy as np
import re
# from unidecode import unidecode
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md', disable=["ner", "textcat", "entity_ruler", "merge_noun_chunks", "merge_entities", "merge_subtokens"])

In [2]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [3]:
# dfExoplanetsDoc=pd.read_pickle("/Users/gerdgrasshoff/Dropbox/temp/df_doc.pcl")
dfExoplanetsDoc=pd.read_pickle("c:/Dropbox/temp/df_doc.pcl")

In [4]:
len(dfExoplanetsDoc)

29745

In [5]:
sentences = []
for doc in dfExoplanetsDoc:
    sent = []
    for sent in doc.sents:
              sentences.append(sent.string.strip())

In [6]:
len(sentences)

293941

In [24]:
dfExoplanetsDoc[40]

M stars are ideal targets to search for Earth-like planets. However, they usually have high levels of magnetic activity, which could affect their habitability and make difficult the detection of exoplanets orbiting around them. Unfortunately, long-term variability of dM stars has not been extensively studied, due to their low intrinsic brightness. For this reason, in 1999 we started the HKα project, which systematically observes the spectra of a large number of stars, in particular dM stars, at the Complejo Astronómico El Leoncito (CASLEO). In this work, we study the long-term activity of the young active dM1 star AU Microscopii. We analyse the Mount Wilson index S derived from CASLEO spectra obtained between 2004 and 2016, which we complement with the S-index derived from HARPS, FEROS, and UVES public spectra. We also analyse the simultaneous photometric counterpart provided by the ASAS public data base for this star between 2000 and 2009, and our own photometry. In both totally indep

In [8]:
def showsentence(nrdoc,nr):
    s=list(dfExoplanetsDoc[nrdoc].sents)
    return(s[nr])
def splitsent(doc,nr):
    return(list(doc.sents)[nr])
def lsent(doc):
    return(list(doc.sents))

In [9]:
s=lsent(dfExoplanetsDoc[10])
s

[We carry out 3D hydrodynamical simulations to study planet-disc interactions for inclined high-mass planets, focusing on the disc's secular evolution induced by the planet.,
 We find that, when the planet is massive enough and the induced gap is deep enough, the disc inside the planet's orbit breaks from the outer disc.,
 The inner and outer discs precess around the system's total angular momentum vector independently at different precession rates, which causes significant disc misalignment.,
 We derive the analytical formulae, which are also verified numerically, for: (1) the relationship between the planet mass and the depth/width of the induced gap, (2) the migration and inclination damping rates for massive inclined planets, and (3) the condition under which the inner and outer discs can break and undergo differential precession.,
 Then, we carry out Monte Carlo radiative transfer calculations for the simulated broken discs.,
 Both disc shadowing in near-infrared images and gas ki

# Dependency label
https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md

https://stackoverflow.com/questions/40288323/what-do-spacys-part-of-speech-and-dependency-tags-mean/40288324

https://nlp.stanford.edu/software/dependencies_manual.pdf

In [10]:
satz=s[1]
satz

We find that, when the planet is massive enough and the induced gap is deep enough, the disc inside the planet's orbit breaks from the outer disc.

displacy.server(satz, style="dep")

In [15]:
[(t.text,t.pos_,t.dep_,t.head.text,t.head.pos_,[c for c in t.children]) for t in satz]

[('We', 'PROPN', 'number', 'find', 'X', []),
 ('find', 'X', 'ROOT', 'find', 'X', [We, breaks, .]),
 ('that', 'ADV', 'nmod', 'breaks', 'X', []),
 (',', 'SCONJ', 'root', 'breaks', 'X', []),
 ('when', 'AUX', 'appos', 'is', 'X', []),
 ('the', 'INTJ', 'hmod', 'planet', 'NUM', []),
 ('planet', 'NUM', 'number', 'is', 'X', [the]),
 ('is', 'X', 'amod', 'breaks', 'X', [when, planet, massive, and, is]),
 ('massive', 'ADP', 'agent', 'is', 'X', [enough]),
 ('enough', 'AUX', 'appos', 'massive', 'ADP', []),
 ('and', 'DET', 'conj', 'is', 'X', []),
 ('the', 'INTJ', 'hmod', 'gap', 'NUM', []),
 ('induced', 'X', 'aux', 'gap', 'NUM', []),
 ('gap', 'NUM', 'number', 'is', 'X', [the, induced]),
 ('is', 'X', 'csubjpass', 'is', 'X', [gap, enough]),
 ('deep', 'ADP', 'appos', 'enough', 'AUX', []),
 ('enough', 'AUX', 'agent', 'is', 'X', [deep]),
 (',', 'SCONJ', 'root', 'breaks', 'X', []),
 ('the', 'INTJ', 'hmod', 'disc', 'NUM', []),
 ('disc', 'NUM', 'number', 'breaks', 'X', [the, inside]),
 ('inside', 'ADV', 'quan

In [16]:
def phil_satz(satz):
    merkmal = dict({"satz":satz.text})
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
#    print("root:",merkmal["act"])
    for t in satz:
#        print("CHECK",t.text,"             ATT=",t.dep_,t.head.text,t.head.pos_)
        if t.dep_ == "dobj" and t.head.pos_ == "VERB":
            merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})
        if t.dep_ in ("ccomp"):
            merkmal.update({"ccomp":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ in ("acomp"):
            merkmal.update({"acomp":''.join(w.text_with_ws for w in t.subtree)})

    return(merkmal)
phil_satz(satz)

{'satz': "We find that, when the planet is massive enough and the induced gap is deep enough, the disc inside the planet's orbit breaks from the outer disc.",
 'act': 'find'}

In [17]:
l=[]
df=pd.DataFrame(columns=["satz","subject","act","ccomp","acomp"])

In [25]:
for i in range(1000):
    s=sentences[i]
    p=phil_satz(nlp(s))
    df=df.append(p,ignore_index=True)

In [19]:
pd.set_option('max_colwidth', 260)
df.head(20)

,satz,subject,act,ccomp,acomp,obj
0,NSVS 10441882 is a newly discovered eclipsing binary system with strong magnetic activity and an orbital period of ∼ 0.5166 days.,nsvs,is,and,NaN,NaN
1,"In order to study this eclipsing binary system, we analyzed its first four-color (BVR<SUB>",we,analyzed,NaN,NaN,its first four-color (BVR<SUB>
2,c</SUB,NaN,c</SUB,NaN,NaN,NaN
3,>,NaN,>,NaN,NaN,NaN
4,I<SUB,NaN,SUB,NaN,NaN,NaN
5,>,NaN,>,NaN,NaN,NaN
6,c</SUB>),NaN,>,NaN,NaN,NaN
7,light curves.,NaN,curves,NaN,NaN,NaN
8,"The observed light curves were asymmetric, so we used the 2013 version of the Wilson-Devinney (W-D) program with cool star-spots to analyze these data.",curves,were,NaN,asymmetric,these data
9,We discovered that NSVS 10441882 is a detached total eclipsing binary system with an orbit inclination of 85°.34 ± 0.07 and a mass ratio of q = 0.94 ± 0.03.,NaN,±,and,NaN,NaN


In [21]:
dfs=df[df["subject"]=="we"]
dfs.head(4)

,satz,subject,act,ccomp,acomp,obj
1,"In order to study this eclipsing binary system, we analyzed its first four-color (BVR<SUB>",we,analyzed,NaN,NaN,its first four-color (BVR<SUB>
15,"In this paper, we consider the collinear symmetric four-body problem, where four masses m",we,consider,NaN,NaN,"the collinear symmetric four-body problem, where four masses m"
21,We introduce regularized variables to deal with binary collisions as well as McGehee coordinates to study the quadruple collision manifold for a negative value of the energy.,we,introduce,as well as,NaN,the quadruple collision manifold
24,"We use high order parametrizations of their stable/unstable manifolds to devise a numerical procedure to compute ejection-collision orbits, for any value of α.",we,use,NaN,NaN,ejection-collision orbits


In [22]:
dfAct=df.groupby(["subject","act"]).size().sort_values(ascending=False).reset_index()
dfAct[dfAct["subject"]=="we"]

,subject,act,0
0,we,present,24
1,we,find,17
2,we,use,10
3,we,report,9
4,we,show,8
6,we,discuss,6
7,we,conclude,6
8,we,derive,5
9,we,identify,5
10,we,investigate,5
